In [1]:
import glob
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn import neighbors

In [6]:
folder = "stack2nd/"
oof = pd.read_feather("oof2_all_test.feather")
test = pd.read_feather("test2_all_test.feather")
traindex = oof.item_id

x = oof.drop(["item_id", "y"], axis=1).values
y = oof.y.values
test_id = test.item_id.values
test_x = test.drop(["item_id"], axis=1).values

In [7]:
x.shape

(1503424, 64)

In [8]:
#ridge
result = []
pred_test = np.zeros((len(test_x)))
train_roc, val_roc = [], []

ans = pd.DataFrame(columns=["item_id", "deal_probability"])
folds = pd.read_csv("data/cvFolds.csv")

for j in range(1, 8):
    train_idx = folds[folds.fold!=j].index.values
    test_idx = folds[folds.fold==j].index.values
    xtr = x[train_idx]
    ytr = y[train_idx]
    xval = x[test_idx]
    yval = y[test_idx]
    val_id = traindex[test_idx]
    
    model = Ridge(alpha=1e-5)
    model.fit(xtr, ytr)
    print(model.coef_)
    
    pred_val = model.predict(xval)
    pred_val = np.clip(pred_val, 0.0, 1.0)
    #pred_val = np.where(pred_val < 0.005, 0, pred_val)
    loss_val = np.sqrt(mean_squared_error(yval, pred_val))
    ans_ = pd.DataFrame({"item_id": val_id, "deal_probability": pred_val})
    ans = pd.concat((ans, ans_), axis=0)
    
    pred_test += np.clip(model.predict(test_x), 0.0, 1.0)
    val_roc.append(loss_val)
    print("test loss", loss_val)

print("mean valloss", np.mean(val_roc))

ans.to_csv(folder+"ridge1_cvPreds_all.csv", index=False)

sub = pd.DataFrame()
sub["item_id"] = test_id
sub["deal_probability"] = pred_test/7
sub["deal_probability"] .clip(0.0, 1.0, inplace=True)
sub.to_csv(folder+"ridge1_testPreds_all.csv", index=False)

[-0.03648105  0.06671315 -0.04710182 -0.10520438  0.00882287  0.01006139
  0.2428907   0.0345227  -0.04119693 -0.01779218 -0.03516799 -0.0013355
 -0.09593122 -0.01710561 -0.01907652  0.45139154 -0.04811358 -0.03732844
 -0.03184909 -0.06102298 -0.03458024 -0.02872616 -0.14408183 -0.01255199
 -0.03523598  0.10343103 -0.02048007 -0.35830465  0.06480817 -0.09456323
  0.10595159  0.06014987  0.04061017  0.02433172 -0.11422082 -0.03712398
 -0.06123426 -0.02575121 -0.11387563  0.20715065  0.02091696  0.11912121
  0.03291356  0.19984827  0.25211151  0.14325107  0.13502712  0.2212015
 -0.19719694  0.05515071  0.04657492 -0.04653303  0.01595117  0.0151539
 -0.03938621 -0.03762645  0.09452845 -0.00627108  0.01528965  0.09799383
  0.06669476  0.09448641  0.02392837 -0.06875766]
test loss 0.20897676806386542
[-0.02925252  0.06247082 -0.04388752 -0.10682964  0.00710844  0.00667241
  0.24979745  0.0390078  -0.04351075 -0.01424417 -0.04074515  0.00226832
 -0.08686697 -0.02382923 -0.02368985  0.4568741

In [9]:
#extra tree
params = {"max_depth": 5,
          "min_samples_leaf": 100,
          "max_features": 1.0,
          "n_estimators": 10,
         "random_state": 0}
result = []
pred_test = np.zeros((len(test_x)))
train_roc, val_roc = [], []
ans = pd.DataFrame(columns=["item_id", "deal_probability"])
folds = pd.read_csv("data/cvFolds.csv")

for j in range(1, 8):
    train_idx = folds[folds.fold!=j].index.values
    test_idx = folds[folds.fold==j].index.values
    xtr = x[train_idx]
    ytr = y[train_idx]
    xval = x[test_idx]
    yval = y[test_idx]
    val_id = traindex[test_idx]
    
    model = ExtraTreesRegressor(**params)
    model.fit(xtr, ytr)
    
    pred_val = model.predict(xval)
    pred_val = np.clip(pred_val, 0.0, 1.0)
    #pred_val = np.where(pred_val < 0.005, 0, pred_val)
    loss_val = np.sqrt(mean_squared_error(yval, pred_val))
    ans_ = pd.DataFrame({"item_id": val_id, "deal_probability": pred_val})
    ans = pd.concat((ans, ans_), axis=0)
    
    pred_test += np.clip(model.predict(test_x), 0.0, 1.0)
    val_roc.append(loss_val)
    print("test loss", loss_val)

print("mean valloss", np.mean(val_roc))

ans.to_csv(folder+"exttree1_cvPreds_all_10.csv", index=False)

sub = pd.DataFrame()
sub["item_id"] = test_id
sub["deal_probability"] = pred_test/7
sub["deal_probability"] .clip(0.0, 1.0, inplace=True)
sub.to_csv(folder+"exttree1_testPreds_all_10.csv", index=False)

test loss 0.21063318145558974
test loss 0.21110201524620142
test loss 0.21108758374981984
test loss 0.21105918924018377
test loss 0.21066529031168987
test loss 0.21118129877943395
test loss 0.21131949660528926
mean valloss 0.21100686505545824


In [5]:
#lgbm
lgbm_params =  {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        "boost_from_average": False,
        #'max_depth': 2,
        'num_leaves': 73,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'feature_fraction': 0.5,
        'min_data_in_leaf': 50,
        #'reg_alpha': 0.6,
        #'reg_lambda': 0.4,
        'min_sum_hessian_in_leaf': 2.0,
        'learning_rate': 0.01,
        'seed': 0,
        'verbose': -1
    }

result = []
pred_test = np.zeros((len(test_x)))
train_roc, val_roc = [], []
ans = pd.DataFrame(columns=["item_id", "deal_probability"])
folds = pd.read_csv("data/cvFolds.csv")

for j in range(1, 8):
    train_idx = folds[folds.fold!=j].index.values
    test_idx = folds[folds.fold==j].index.values
    xtr = x[train_idx]
    ytr = y[train_idx]
    xval = x[test_idx]
    yval = y[test_idx]
    val_id = traindex[test_idx]
    
    lgtrain = lgb.Dataset(xtr, ytr)
    lgvalid = lgb.Dataset(xval, yval)

    lgb_clf = lgb.train(
        lgbm_params,
        lgtrain,
        num_boost_round=100000,
        valid_sets=[lgtrain, lgvalid],
        valid_names=['train','valid'],
        early_stopping_rounds=400,
        verbose_eval=200
    )
    
    pred_val = lgb_clf.predict(xval, num_iteration=lgb_clf.best_iteration)
    pred_val = np.clip(pred_val, 0.0, 1.0)
    loss_val = np.sqrt(mean_squared_error(yval, pred_val))
    print("test loss", loss_val)
    
    pred_test += np.clip(lgb_clf.predict(test_x, num_iteration=lgb_clf.best_iteration), 0.0, 1.0)
    val_roc.append(loss_val)
    
    ans_ = pd.DataFrame({"item_id": val_id, "deal_probability": pred_val})
    ans = pd.concat((ans, ans_), axis=0)

print("mean valloss", np.mean(val_roc))

ans.to_csv(folder+"lgbm1_cvPreds_all.csv", index=False)

sub = pd.DataFrame()
sub["item_id"] = test_id
sub["deal_probability"] = pred_test/7
sub["deal_probability"] .clip(0.0, 1.0, inplace=True)
sub.to_csv(folder+"lgbm1_testPreds_all.csv", index=False)

Training until validation scores don't improve for 400 rounds.
[200]	train's rmse: 0.211221	valid's rmse: 0.211257
[400]	train's rmse: 0.20842	valid's rmse: 0.209041
[600]	train's rmse: 0.207703	valid's rmse: 0.208874
[800]	train's rmse: 0.207133	valid's rmse: 0.208809
[1000]	train's rmse: 0.206636	valid's rmse: 0.208767
[1200]	train's rmse: 0.206153	valid's rmse: 0.208734
[1400]	train's rmse: 0.205692	valid's rmse: 0.208715
[1600]	train's rmse: 0.205252	valid's rmse: 0.208698
[1800]	train's rmse: 0.204825	valid's rmse: 0.208687
[2000]	train's rmse: 0.20441	valid's rmse: 0.208679
[2200]	train's rmse: 0.203997	valid's rmse: 0.208675
[2400]	train's rmse: 0.203578	valid's rmse: 0.20867
[2600]	train's rmse: 0.20318	valid's rmse: 0.208671
[2800]	train's rmse: 0.202793	valid's rmse: 0.208662
[3000]	train's rmse: 0.202408	valid's rmse: 0.208665
[3200]	train's rmse: 0.202024	valid's rmse: 0.208661
[3400]	train's rmse: 0.201651	valid's rmse: 0.208654
[3600]	train's rmse: 0.201272	valid's rmse: 

KeyboardInterrupt: 

In [4]:
#xgb
xgb_params =  {
        'task': 'train',
        'booster': 'gblinear',
        'objective': 'reg:linear',
        'metric': 'rmse',
        'max_depth': 5,
        'max_leaves': 31,
        'subsample': 0.8,
        'colsample_bylevel': 0.5,
        'min_child_weight': 100,
        'eta': 0.05,
        'silent': 1,
        'verbose': 0
    }  

result = []
pred_test = np.zeros((len(test_x)))
train_roc, val_roc = [], []
ans = pd.DataFrame(columns=["item_id", "deal_probability"])
folds = pd.read_csv("data/cvFolds.csv")

for j in range(1, 8):
    train_idx = folds[folds.fold!=j].index.values
    test_idx = folds[folds.fold==j].index.values
    xtr = x[train_idx]
    ytr = y[train_idx]
    xval = x[test_idx]
    yval = y[test_idx]
    val_id = traindex[test_idx]
    
    dtrain =xgb.DMatrix(data = xtr, label = ytr)
    dval =xgb.DMatrix(data = xval, label = yval)
    watchlist = [(dval, 'eval')]
    
    xgb_clf = xgb.train(
                            xgb_params,
                            dtrain,
                            num_boost_round=15000,
                            early_stopping_rounds=400,
                            verbose_eval=1000,
                            evals=watchlist
                            )
    
    pred_val = xgb_clf.predict(dval, ntree_limit=xgb_clf.best_ntree_limit)
    pred_val = np.clip(pred_val, 0.0, 1.0)
    loss_val = np.sqrt(mean_squared_error(yval, pred_val))
    
    dtest =xgb.DMatrix(data = test_x)
    pred_test += np.clip(xgb_clf.predict(dtest, ntree_limit=xgb_clf.best_ntree_limit), 0.0, 1.0)
    val_roc.append(loss_val)
    
    ans_ = pd.DataFrame({"item_id": val_id, "deal_probability": pred_val})
    ans = pd.concat((ans, ans_), axis=0)
    
    print("test loss", loss_val)

print("mean valloss", np.mean(val_roc))

ans.to_csv(folder+"xgblinear1_cvPreds.csv", index=False)

sub = pd.DataFrame()
sub["item_id"] = test_id
sub["deal_probability"] = pred_test/7
sub["deal_probability"] .clip(0.0, 1.0, inplace=True)
sub.to_csv(folder+"xgblinear1_testPreds.csv", index=False)

[0]	eval-rmse:0.411432
Will train until eval-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[339]	eval-rmse:0.212982

test loss 0.21298231988278543
[0]	eval-rmse:0.411475
Will train until eval-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[196]	eval-rmse:0.213282

test loss 0.2132821498074323
[0]	eval-rmse:0.411506
Will train until eval-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[289]	eval-rmse:0.212495

test loss 0.21249525848758216
[0]	eval-rmse:0.411466
Will train until eval-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[137]	eval-rmse:0.213482

test loss 0.21348192273251976
[0]	eval-rmse:0.411597
Will train until eval-rmse hasn't improved in 10 rounds.
Stopping. Best iteration:
[255]	eval-rmse:0.212076

test loss 0.21207586167563877
mean valloss 0.21286350251719172


In [ ]:
#knn uni
params = {"n_neighbors": 5,
          "weights": "uniform",
          "leaf_size": 30}
result = []
pred_test = np.zeros((len(test_x)))
train_roc, val_roc = [], []
ans = pd.DataFrame(columns=["item_id", "deal_probability"])
folds = pd.read_csv("data/cvFolds.csv")

for j in range(1, 8):
    train_idx = folds[folds.fold!=j].index.values
    test_idx = folds[folds.fold==j].index.values
    xtr = x[train_idx]
    ytr = y[train_idx]
    xval = x[test_idx]
    yval = y[test_idx]
    val_id = traindex[test_idx]
    
    model = neighbors.KNeighborsRegressor(**params)
    model.fit(xtr, ytr)
    
    pred_val = model.predict(xval)
    pred_val = np.clip(pred_val, 0.0, 1.0)
    #pred_val = np.where(pred_val < 0.005, 0, pred_val)
    loss_val = np.sqrt(mean_squared_error(yval, pred_val))
    ans_ = pd.DataFrame({"item_id": val_id, "deal_probability": pred_val})
    ans = pd.concat((ans, ans_), axis=0)
    
    pred_test += np.clip(model.predict(test_x), 0.0, 1.0)
    val_roc.append(loss_val)
    print("test loss", loss_val)

print("mean valloss", np.mean(val_roc))

ans.to_csv(folder+"knnuni1_cvPreds.csv", index=False)

sub = pd.DataFrame()
sub["item_id"] = test_id
sub["deal_probability"] = pred_test/7
sub["deal_probability"] .clip(0.0, 1.0, inplace=True)
sub.to_csv(folder+"knnuni1_textPreds.csv", index=False)